In [1]:
import sys
import ase
import nglview as nv
import numpy as np

sys.path.append('../')


In [41]:
generated_paths = {
    # "Symphony (train)": "/Users/ameyad/Documents/spherical-harmonic-net/conditional_generation/train/qm9_bessel_embedding_attempt6_edm_splits/e3schnet_and_nequip/interactions=3/l=5/position_channels=2/channels=64/fait=1.0/pit=1.0/step=4950000/molecules/",
    # "Symphony (test)": "/Users/ameyad/Documents/spherical-harmonic-net/conditional_generation/test/qm9_bessel_embedding_attempt6_edm_splits/e3schnet_and_nequip/interactions=3/l=5/position_channels=2/channels=64/fait=1.0/pit=1.0/step=4950000/molecules/",
    "GSchNet (train)": "/Users/songk/atomicarchitects/generated_iclr_train",
    "GSchNet (test)": "/Users/songk/atomicarchitects/generated_iclr_test",
}

In [3]:
from analyses.metrics import *

/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/miniconda3/envs/sh-net/lib/python3.10/site-packages/flax/struct.py:136: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/songk/atomicarchitects/spherical_harmonic_net/analyses/../analyses/edm_analyses/data/prepare/process.py:136: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if line[0] is "#":
/Users/songk/atomicarchitects/spherical_harmonic_net/analyses/../analyses/edm_analyses/data/prepare/process.py:138: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if line_counter is 0:
/Users/songk/atomicarchitects

In [42]:
from symphony.data import qm9
all_mols = qm9.load_qm9("../qm9_data", use_edm_splits=True, check_molecule_sanity=False)
num_mols = 1000
test_mols = all_mols[-num_mols:]
train_mols = all_mols[:num_mols]

def get_fragment_list(mols: Sequence[ase.Atoms], num_mols: int):
    fragments = []
    H_removed = []
    for i in range(num_mols):
        mol = mols[i]
        num_atoms = len(mol)
        for j in range(num_atoms):
            fragment = ase.Atoms(
                positions=np.vstack([mol.positions[:j], mol.positions[j + 1 :]]),
                numbers=np.concatenate([mol.numbers[:j], mol.numbers[j + 1 :]]),
            )
            fragments.append(fragment)
            H_removed.append(mol.numbers[j] == 1)
    return fragments, H_removed

# Ensure that the number of molecules is a multiple of num_seeds_per_chunk.
train_mol_list, train_H_removed = get_fragment_list(train_mols, num_mols)
test_mol_list, test_H_removed = get_fragment_list(test_mols, num_mols)

[14:55:00] Warning: molecule is tagged as 3D, but all Z coords are zero
[14:55:00] Warning: molecule is tagged as 3D, but all Z coords are zero


../qm9_data/edm_splits.npz


In [44]:
# def key_fn(x):
#     return int(x.removesuffix(".xyz").split("=")[-1])

def key_fn(x):
    '''oops'''
    tokens = x.removesuffix(".xyz").split('_')
    return (int(tokens[1][3:]), int(tokens[2]))


def get_all_molecules(molecules_dir: str) -> List[Chem.Mol]:
    """Returns all molecules in a directory."""
    molecules = []
    for molecules_file in sorted([f for f in os.listdir(molecules_dir) if f.endswith(".xyz")], key=key_fn):
        if not molecules_file.endswith(".xyz"):
            continue

        molecules_file_full = os.path.join(molecules_dir, molecules_file)
        mol = xyz_to_rdkit_molecule(molecules_file_full)
        molecules.append(mol)

    return molecules


all_generated_molecules = {
    model: get_all_molecules(molecules_dir) for model, molecules_dir in generated_paths.items()
}

['generated', 'src972', '1982']
['generated', 'src258', '279']
['generated', 'src543', '1073']
['generated', 'src604', '1510']
['generated', 'src815', '1595']
['generated', 'src5', '11']
['generated', 'src746', '1423']
['generated', 'src679', '531']
['generated', 'src2', '18']
['generated', 'src879', '1286']
['generated', 'src509', '1342']
['generated', 'src536', '171']
['generated', 'src511', '1167']
['generated', 'src907', '769']
['generated', 'src36', '102']
['generated', 'src193', '311']
['generated', 'src162', '533']
['generated', 'src581', '1478']
['generated', 'src469', '348']
['generated', 'src14', '22']
['generated', 'src193', '305']
['generated', 'src980', '2308']
['generated', 'src641', '1210']
['generated', 'src509', '1356']
['generated', 'src864', '914']
['generated', 'src21', '99']
['generated', 'src511', '1173']
['generated', 'src815', '1581']
['generated', 'src594', '473']
['generated', 'src183', '311']
['generated', 'src409', '1058']
['generated', 'src746', '1437']
['g

ValueError: invalid literal for int() with base 10: 'tensor(743)'

In [36]:
print(np.mean(test_H_removed))

0.512976592330253


In [ ]:
for model, generated_molecules in all_generated_molecules.items():
    if "train" in model:
        H_removed = train_H_removed
    else:
        H_removed = test_H_removed

    generated_molecules_with_H_removed = []
    for i, molecule in enumerate(generated_molecules):
        if H_removed[i]:
            generated_molecules_with_H_removed.append(molecule)
    
    validity_fraction = compute_validity(generated_molecules_with_H_removed, get_all_valid_molecules(generated_molecules_with_H_removed))
    print(f"{model}: {100 * validity_fraction:0.2f}")

GSchNet (test): 96.33


In [99]:
all_generated_molecules["Symphony (train)"][0]

In [100]:
generated_molecules_with_H_removed = []
true_molecules_with_H_removed = []
for generated_molecule, true_molecule, H_removed in zip(all_generated_molecules["Symphony (train)"], train_mol_list, train_H_removed):
    if H_removed:
        generated_molecules_with_H_removed.append(generated_molecule)
        true_molecules_with_H_removed.append(true_molecule)

print(generated_molecules_with_H_removed[0], true_molecules_with_H_removed[0])
ase_mols = [
    ase.Atoms(
        symbols=[atom.GetSymbol() for atom in generated_molecules_with_H_removed[0].GetAtoms()],
        positions=generated_molecules_with_H_removed[0].GetConformer().GetPositions(),
    ),
    true_molecules_with_H_removed[0]]

nv.show_ase(ase_mols[0])


<rdkit.Chem.rdchem.Mol object at 0x336685df0> Atoms(symbols='C3FN2OCNH3', pbc=False)


NGLWidget()

In [101]:
nv.show_ase(ase_mols[1])

NGLWidget()

In [57]:
invalid_molecules_symphony = [mol for mol in all_generated_molecules["Symphony (train)"][:100] if not check_molecule_validity(mol)]
invalid_molecules_gschnet = [mol for mol in all_generated_molecules["GSchNet (train)"][:100] if not check_molecule_validity(mol)]

In [58]:
invalid_molecules_ase = [
    ase.Atoms(
        symbols=[atom.GetSymbol() for atom in mol.GetAtoms()],
        positions=mol.GetConformer().GetPositions(),
    )
    for mol in invalid_molecules_symphony
]
nv.show_ase(invalid_molecules_ase[2], gui=True)        

NGLWidget()